# F1 Pit Stop Feature Engineering

**Goal:** Transform raw pit stop data into ML-ready features

**Input:** `data/raw/f1_2023_pit_stops.csv` (930 pit stops)

**Output:** `data/processed/ml_features.csv` (with engineered features)

**Features we'll create:**
1. **Tire age** - How many laps on current compound
2. **Stop number** - Is this 1st, 2nd, or 3rd pit stop?
3. **Race progress** - What % of race is complete?
4. **Stint length** - How long between pit stops
5. **Position changes** - Gaining or losing places?


In [26]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [27]:
df = pd.read_csv('data/raw/f1_2023_pit_stops.csv')

print(f"dataset loaded")
print(f" Rows: {len(df)}")
print(f" Columns: {len(df.columns)}")
print(f"\n cloumn names:")
print (df.columns.tolist())

print("\n sample data:")
df.head()

dataset loaded
 Rows: 930
 Columns: 33

 cloumn names:
['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason', 'FastF1Generated', 'IsAccurate', 'race_round', 'RaceName']

 sample data:


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,race_round,RaceName
0,0 days 01:27:33.930000,VER,1,0 days 00:01:58.378000,15.0,2.0,0 days 01:25:58.127000,NaN,0 days 00:00:51.854000,0 days 00:00:42.629000,0 days 00:00:23.895000,0 days 01:26:27.465000,0 days 01:27:10.094000,0 days 01:27:33.989000,230.0,237.0,279.0,218.0,False,SOFT,1.0,False,Red Bull Racing,0 days 01:25:35.552000,2023-03-05 15:26:37.401,1,2.0,False,NaN,False,False,1,Bahrain Grand Prix
1,0 days 02:03:45.820000,VER,1,0 days 00:01:58.490000,37.0,3.0,0 days 02:02:10.556000,NaN,0 days 00:00:52.755000,0 days 00:00:42.071000,0 days 00:00:23.664000,0 days 02:02:40.144000,0 days 02:03:22.215000,0 days 02:03:45.879000,232.0,244.0,281.0,215.0,False,HARD,1.0,True,Red Bull Racing,0 days 02:01:47.330000,2023-03-05 16:02:49.179,1,1.0,False,NaN,False,False,1,Bahrain Grand Prix
2,0 days 01:19:58.205000,GAS,10,0 days 00:02:01.262000,10.0,2.0,0 days 01:18:21.104000,NaN,0 days 00:00:54.574000,0 days 00:00:42.660000,0 days 00:00:24.028000,0 days 01:18:51.526000,0 days 01:19:34.186000,0 days 01:19:58.214000,227.0,245.0,278.0,215.0,False,HARD,1.0,True,Alpine,0 days 01:17:56.943000,2023-03-05 15:18:58.792,1,20.0,False,NaN,False,False,1,Bahrain Grand Prix
3,0 days 01:46:47.684000,GAS,10,0 days 00:01:59.404000,26.0,3.0,0 days 01:45:11.481000,NaN,0 days 00:00:53.105000,0 days 00:00:42.400000,0 days 00:00:23.899000,0 days 01:45:41.394000,0 days 01:46:23.794000,0 days 01:46:47.693000,228.0,250.0,280.0,215.0,False,HARD,1.0,True,Alpine,0 days 01:44:48.280000,2023-03-05 15:45:50.129,1,17.0,False,NaN,False,False,1,Bahrain Grand Prix
4,0 days 02:12:01.223000,GAS,10,0 days 00:02:08.544000,41.0,4.0,0 days 02:10:15.719000,NaN,0 days 00:01:00.744000,0 days 00:00:44.327000,0 days 00:00:23.473000,0 days 02:10:53.432000,0 days 02:11:37.759000,0 days 02:12:01.232000,169.0,255.0,289.0,208.0,False,SOFT,1.0,True,Alpine,0 days 02:09:52.679000,2023-03-05 16:10:54.528,671,10.0,False,NaN,False,False,1,Bahrain Grand Prix


In [28]:
key_columns = ['Driver', 'LapNumber', 'LapTime', 'TyreLife', 
               'Compound', 'FreshTyre', 'Position', 'Stint', 
               'TrackStatus', 'RaceName', 'race_round', 'SpeedFL']

df_features = df[key_columns].copy()

print("🔍 Key columns for ML (with TrackStatus!):")
print(df_features.head(10))

print(f"\n📊 Data types:")
print(df_features.dtypes)

print(f"\n❓ Missing values:")
print(df_features.isnull().sum())

print(f"\n🔍 FreshTyre distribution:")
print(df_features['FreshTyre'].value_counts())

print(f"\n🚦 TrackStatus distribution:")
print(df_features['TrackStatus'].value_counts())

🔍 Key columns for ML (with TrackStatus!):
  Driver  LapNumber                 LapTime  TyreLife Compound  FreshTyre  \
0    VER       15.0  0 days 00:01:58.378000       1.0     SOFT      False   
1    VER       37.0  0 days 00:01:58.490000       1.0     HARD       True   
2    GAS       10.0  0 days 00:02:01.262000       1.0     HARD       True   
3    GAS       26.0  0 days 00:01:59.404000       1.0     HARD       True   
4    GAS       41.0  0 days 00:02:08.544000       1.0     SOFT       True   
5    PER       18.0  0 days 00:01:58.797000       3.0     SOFT      False   
6    PER       35.0  0 days 00:01:58.785000       1.0     HARD       True   
7    ALO       15.0  0 days 00:01:59.692000       1.0     HARD       True   
8    ALO       35.0  0 days 00:01:57.389000       1.0     HARD       True   
9    LEC       14.0  0 days 00:01:58.770000       1.0     HARD       True   

   Position  Stint  TrackStatus            RaceName  race_round  SpeedFL  
0       2.0    2.0            1  Ba

In [29]:
# How many rows are complete (no missing values)?
print("📊 Missing Data Analysis:")
print(f"   Total rows: {len(df_features)}")
print(f"   Rows with missing LapTime: {df_features['LapTime'].isnull().sum()}")
print(f"   Rows with missing SpeedFL: {df_features['SpeedFL'].isnull().sum()}")

# How many rows would we lose if we drop missing values?
complete_rows = df_features.dropna()
print(f"\n   Complete rows (no missing): {len(complete_rows)}")
print(f"   Rows we'd lose: {len(df_features) - len(complete_rows)}")
print(f"   Percentage kept: {len(complete_rows)/len(df_features)*100:.1f}%")

# Let's keep the complete rows for now
df_clean = df_features.dropna().copy()

print(f"\n✅ Created clean dataset with {len(df_clean)} rows")

📊 Missing Data Analysis:
   Total rows: 930
   Rows with missing LapTime: 176
   Rows with missing SpeedFL: 58

   Complete rows (no missing): 725
   Rows we'd lose: 205
   Percentage kept: 78.0%

✅ Created clean dataset with 725 rows


In [30]:
# First, check what data type LapTime is
print(f"🔍 LapTime data type: {df_clean['LapTime'].dtype}")
print(f"🔍 Sample LapTime values:")
print(df_clean['LapTime'].head())

# Convert LapTime to timedelta if it's a string
df_clean['LapTime'] = pd.to_timedelta(df_clean['LapTime'])

print(f"\n✅ Converted to timedelta!")
print(f"   New data type: {df_clean['LapTime'].dtype}")

# NOW we can use .dt.total_seconds()
df_clean['LapTime_seconds'] = df_clean['LapTime'].dt.total_seconds()

print(f"\n🔄 Converted to seconds!")
print(f"\n🔍 Sample conversions:")
print(df_clean[['LapTime', 'LapTime_seconds']].head())

print(f"\n📊 LapTime statistics (in seconds):")
print(df_clean['LapTime_seconds'].describe())

# Drop the original LapTime column
df_clean = df_clean.drop('LapTime', axis=1)

print(f"\n✅ LapTime converted and original column dropped!")

🔍 LapTime data type: object
🔍 Sample LapTime values:
0    0 days 00:01:58.378000
1    0 days 00:01:58.490000
2    0 days 00:02:01.262000
3    0 days 00:01:59.404000
4    0 days 00:02:08.544000
Name: LapTime, dtype: object

✅ Converted to timedelta!
   New data type: timedelta64[ns]

🔄 Converted to seconds!

🔍 Sample conversions:
                 LapTime  LapTime_seconds
0 0 days 00:01:58.378000          118.378
1 0 days 00:01:58.490000          118.490
2 0 days 00:02:01.262000          121.262
3 0 days 00:01:59.404000          119.404
4 0 days 00:02:08.544000          128.544

📊 LapTime statistics (in seconds):
count    725.000000
mean     110.418007
std       14.685606
min       78.165000
25%      100.678000
50%      109.850000
75%      119.931000
max      149.911000
Name: LapTime_seconds, dtype: float64

✅ LapTime converted and original column dropped!


In [31]:
# First, let's see what TrackStatus looks like
print("🚦 TrackStatus sample values:")
print(df_clean['TrackStatus'].head(20))

print(f"\n🚦 Unique TrackStatus values:")
print(df_clean['TrackStatus'].value_counts().head(15))

# Convert TrackStatus to string (in case it's not)
df_clean['TrackStatus'] = df_clean['TrackStatus'].astype(str)

# Create binary features for each status type
df_clean['has_safety_car'] = df_clean['TrackStatus'].str.contains('4', na=False).astype(int)
df_clean['has_vsc'] = df_clean['TrackStatus'].str.contains('6', na=False).astype(int)
df_clean['has_red_flag'] = df_clean['TrackStatus'].str.contains('5', na=False).astype(int)
df_clean['has_yellow'] = df_clean['TrackStatus'].str.contains('2', na=False).astype(int)

print("\n✅ Created track status features!")
print(f"\n📊 Pit stops under different conditions:")
print(f"   Safety Car (4): {df_clean['has_safety_car'].sum()} pit stops ({df_clean['has_safety_car'].sum()/len(df_clean)*100:.1f}%)")
print(f"   VSC (6): {df_clean['has_vsc'].sum()} pit stops ({df_clean['has_vsc'].sum()/len(df_clean)*100:.1f}%)")
print(f"   Red Flag (5): {df_clean['has_red_flag'].sum()} pit stops ({df_clean['has_red_flag'].sum()/len(df_clean)*100:.1f}%)")
print(f"   Yellow Flag (2): {df_clean['has_yellow'].sum()} pit stops ({df_clean['has_yellow'].sum()/len(df_clean)*100:.1f}%)")

# Drop original TrackStatus column
df_clean = df_clean.drop('TrackStatus', axis=1)

print(f"\n✅ Replaced TrackStatus with 4 binary features")
print(f"\n📋 Current columns:")
print(df_clean.columns.tolist())


🚦 TrackStatus sample values:
0       1
1       1
2       1
3       1
4     671
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15    671
16      1
17      1
18    671
19      1
Name: TrackStatus, dtype: int64

🚦 Unique TrackStatus values:
TrackStatus
1      585
4       29
41      28
12      26
671     22
124     12
21      10
64       6
67       3
126      2
167      2
Name: count, dtype: int64

✅ Created track status features!

📊 Pit stops under different conditions:
   Safety Car (4): 75 pit stops (10.3%)
   VSC (6): 35 pit stops (4.8%)
   Red Flag (5): 0 pit stops (0.0%)
   Yellow Flag (2): 50 pit stops (6.9%)

✅ Replaced TrackStatus with 4 binary features

📋 Current columns:
['Driver', 'LapNumber', 'TyreLife', 'Compound', 'FreshTyre', 'Position', 'Stint', 'RaceName', 'race_round', 'SpeedFL', 'LapTime_seconds', 'has_safety_car', 'has_vsc', 'has_red_flag', 'has_yellow']


In [32]:
# Let's check if we can detect red flag tire changes
# Load the ORIGINAL data again to investigate
df_original = pd.read_csv('data/raw/f1_2023_pit_stops.csv')

# Look for races that had red flags
print("🔍 Checking for red flags in original data...")
print(f"   TrackStatus values containing '5' (red flag):")

red_flag_laps = df_original[df_original['TrackStatus'].astype(str).str.contains('5', na=False)]
print(f"   Total laps with red flag: {len(red_flag_laps)}")

if len(red_flag_laps) > 0:
    print(f"\n🚩 Red flags occurred in these races:")
    print(red_flag_laps[['RaceName', 'LapNumber', 'Driver', 'TrackStatus']].head(20))
    
    # Check if TyreLife resets after red flag
    print(f"\n🔍 Checking TyreLife around red flags:")
    print(red_flag_laps[['Driver', 'LapNumber', 'TyreLife', 'FreshTyre', 'TrackStatus']].head(20))
else:
    print("\n✅ No red flags in our 2023 dataset!")
    print("   This explains why has_red_flag = 0")

🔍 Checking for red flags in original data...
   TrackStatus values containing '5' (red flag):
   Total laps with red flag: 41

🚩 Red flags occurred in these races:
                  RaceName  LapNumber Driver  TrackStatus
95   Australian Grand Prix       55.0    SAR           51
97   Australian Grand Prix        8.0    MAG          451
101  Australian Grand Prix       55.0    DEV           51
104  Australian Grand Prix       55.0    TSU          451
109  Australian Grand Prix       55.0    ZHO          451
124  Australian Grand Prix        8.0    SAI          451
128  Australian Grand Prix        8.0    RUS          451
133  Australian Grand Prix       54.0    BOT           45
134  Australian Grand Prix       55.0    BOT           51
138  Australian Grand Prix       55.0    PIA          451
468       Dutch Grand Prix       64.0    VER          265
479       Dutch Grand Prix       64.0    PER          651
493       Dutch Grand Prix       64.0    STR          514
500       Dutch Grand Pr

### Red Flag Investigation Results

**Question:** Do red flags cause data issues with tire tracking?

**Finding:** ✅ No issues! FastF1 correctly tracks tire changes during red flags.

**Evidence:**
- Australian GP & Dutch GP had red flags in 2023
- TyreLife resets to 1.0 when fresh tires fitted during red flag
- FreshTyre accurately indicates new vs old tires
- Strategic variety: Some drivers changed tires, others didn't

**Conclusion:** Existing features (TyreLife, FreshTyre) already capture red flag tire strategy.
No additional features needed.

In [33]:
print("Compound Types:")
print(df_clean['Compound'].value_counts())

compound_mapping = {
    'SOFT': 1,
    'MEDIUM': 2,
    'HARD': 3,
    'INTERMEDIATE': 4,
    'WET': 5
}

df_clean['Compound_encoded'] = df_clean['Compound'].map(compound_mapping)

print("\n✅ Encoded Compound types:")
print ("encoding mapping:")
for compound, code in sorted(compound_mapping.items(), key=lambda x: x[1]):
    count = (df_clean['Compound'] == compound).sum()
    pct = count / len(df_clean) * 100
    print(f"  {code} = {compound:15} ->  {count:3} pit stops ({pct:5.1f}%)")

print ("verification sample:")
print(df_clean[['Compound', 'Compound_encoded']].head(10))

unmapped = df_clean['Compound_encoded'].isnull().sum()

if unmapped > 0:
    print(f"\n⚠️ Warning: {unmapped} compounds not mapped!")
    unmapped_compounds = df_clean[df_clean['Compound_encoded'].isnull()]['Compound'].tolist()
    print(f"   Unmapped values: {unmapped_compounds}")
else:
    print("\n✅ All Compound values successfully mapped!")

# Drop original Compound column
df_clean = df_clean.drop('Compound', axis=1)

print(f"\n✅ Dropped original Compound column")
print(f"\n📋 Final columns: {len(df_clean.columns)}")

Compound Types:
Compound
HARD            348
MEDIUM          204
SOFT            115
INTERMEDIATE     54
WET               4
Name: count, dtype: int64

✅ Encoded Compound types:
encoding mapping:
  1 = SOFT            ->  115 pit stops ( 15.9%)
  2 = MEDIUM          ->  204 pit stops ( 28.1%)
  3 = HARD            ->  348 pit stops ( 48.0%)
  4 = INTERMEDIATE    ->   54 pit stops (  7.4%)
  5 = WET             ->    4 pit stops (  0.6%)
verification sample:
  Compound  Compound_encoded
0     SOFT                 1
1     HARD                 3
2     HARD                 3
3     HARD                 3
4     SOFT                 1
5     SOFT                 1
6     HARD                 3
7     HARD                 3
8     HARD                 3
9     HARD                 3

✅ All Compound values successfully mapped!

✅ Dropped original Compound column

📋 Final columns: 15
